In [1]:
%matplotlib inline
import utils
reload(utils)
from utils import *

Using TensorFlow backend.


In [2]:
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [3]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [4]:
x = random((30, 2))
y = np.dot(x, [2., 3.]) + 1

In [5]:
x[:5]

array([[ 0.3042,  0.6766],
       [ 0.3899,  0.9713],
       [ 0.4704,  0.0831],
       [ 0.1402,  0.8995],
       [ 0.2773,  0.6338]])

In [6]:
y[:5]

array([ 3.6383,  4.6936,  2.19  ,  3.9789,  3.456 ])

In [7]:
lm = Sequential()
lm.add(Dense(1, input_shape = (2,)))
lm.compile(optimizer=SGD(lr = 0.01), loss='mse')

In [8]:
lm.evaluate(x, y, verbose=0)

28.415536880493164

In [9]:
lm.fit(x, y, nb_epoch=20, batch_size = 1)

Epoch 1/20
30/30 [==============================] - 0s - loss: 12.5031      3/30 [==>...........................] - ETA: 1s - loss: 30.2256
Epoch 2/20
30/30 [==============================] - 0s - loss: 1.7977     

In [10]:
lm.evaluate(x, y, verbose=0)

0.13940730690956116

In [11]:
lm.get_weights()

[array([[ 0.913 ],
        [ 1.9228]], dtype=float32), array([ 2.3193], dtype=float32)]

In [12]:
data_path = "/Users/trinakarmakar/anaconda2/data/fastai/redux/sample/"
model_path = data_path + "models/"
if not os.path.exists(model_path):
    os.mkdir(model_path)

In [13]:
batch_size = 100
from Vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [14]:
val_batches = get_batches(data_path+'valid', shuffle=False, batch_size=1)
batches = get_batches(data_path+'train', shuffle=False, batch_size=1)

Found 50 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [15]:
import bcolz
def save_array(fname, arr):
    c = bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)

In [16]:
val_data = get_data(data_path+'valid')
train_data = get_data(data_path+'train')

Found 50 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [17]:
train_data.shape

(200, 3, 224, 224)

In [18]:
save_array(model_path+'train_data.bc', train_data)
save_array(model_path+'val_data.bc', val_data)

In [19]:
train_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'val_data.bc')

In [20]:
print(val_data.shape)

(50, 3, 224, 224)


In [21]:
def onehot(x):
    return np.array(OneHotEncoder().fit_transform(x.reshape(-1, 1)).todense())

In [22]:
val_classes = val_batches.classes
train_classes = batches.classes
val_labels = onehot(val_classes)
train_labels = onehot(train_classes)

In [23]:
train_labels.shape

(200, 2)

In [24]:
train_classes[:4]

array([0, 0, 0, 0], dtype=int32)

In [25]:
train_labels[:4]

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [ ]:
train_features = model.predict(train_data, batch_size=batch_size)
val_features = model.predict(val_data, batch_size=batch_size)